In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1=pd.read_csv('city_day.csv')
df_2=pd.read_csv('city_hour.csv')
df_3=pd.read_csv('station_day.csv')
df_4=pd.read_csv('station_hour.csv')
df_5=pd.read_csv('stations.csv')

C:\Users\ajayb\AppData\Local\Temp\ipykernel_20764\3305233365.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_4=pd.read_csv('station_hour.csv')


In [3]:
df_2['Datetime'] = pd.to_datetime(df_2['Datetime'])

In [4]:
df_2['Date'] = df_2['Datetime'].dt.date

In [5]:
df_2 = df_2.drop(['Datetime'], axis = 1)

In [6]:
merge1 = pd.concat([df_1, df_2], axis = 0)

In [7]:
df_4['Datetime'] = pd.to_datetime(df_4['Datetime'])

In [8]:
df_4['Date'] = df_4['Datetime'].dt.date

In [9]:
df_4 = df_4.drop(['Datetime'], axis = 1)

In [10]:
merge2 = pd.concat([df_3, df_4], axis = 0)

In [11]:
merged3=pd.merge(df_5,merge2,on='StationId')

In [12]:
# concatenating df1 and df2 along rows
df = pd.concat([merge2, merged3], axis=0)

In [13]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [14]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)
    
    def split(self, X, y, idx, split_val):
        left_idx = np.argwhere(X[:, idx] <= split_val).flatten()
        right_idx = np.argwhere(X[:, idx] > split_val).flatten()
        
        if len(left_idx) == 0 or len(right_idx) == 0:
            return None, None
        
        left_X, left_y = X[left_idx], y[left_idx]
        right_X, right_y = X[right_idx], y[right_idx]
        
        left_mse, right_mse = self.mse(left_y), self.mse(right_y)
        return left_X, left_y, right_X, right_y, left_mse + right_mse
    
    def find_best_split(self, X, y):
        best_idx, best_val, best_mse = None, None, np.inf
        
        for idx in range(X.shape[1]):
            for val in np.unique(X[:, idx]):
                left_X, left_y, right_X, right_y, mse = self.split(X, y, idx, val)
                if mse is not None and mse < best_mse:
                    best_idx, best_val, best_mse = idx, val, mse
        
        return best_idx, best_val, best_mse
    
    def fit(self, X, y, depth=0):
        if self.max_depth is not None and depth >= self.max_depth:
            self.left, self.right = None, None
            self.prediction = np.mean(y)
            return
        
        idx, val, mse = self.find_best_split(X, y)
        if idx is None:
            self.left, self.right = None, None
            self.prediction = np.mean(y)
            return
        
        left_X, left_y, right_X, right_y, _ = self.split(X, y, idx, val)
        self.feature_idx = idx
        self.split_val = val
        self.left = DecisionTreeRegressor(max_depth=self.max_depth)
        self.right = DecisionTreeRegressor(max_depth=self.max_depth)
        self.left.fit(left_X, left_y, depth=depth+1)
        self.right.fit(right_X, right_y, depth=depth+1)
    
    def predict(self, X):
        if self.left is None and self.right is None:
            return self.prediction
        
        if X[self.feature_idx] <= self.split_val:
            return self.left.predict(X)
        else:
            return self.right.predict(X)

In [15]:
class RandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for tree in self.trees:
            predictions.append(tree.predict(X))
        return np.mean(predictions, axis=0)

In [16]:
df.dtypes

StationId       object
Date            object
PM2.5          float64
PM10           float64
NO             float64
NO2            float64
NOx            float64
NH3            float64
CO             float64
SO2            float64
O3             float64
Benzene        float64
Toluene        float64
Xylene         float64
AQI            float64
AQI_Bucket      object
StationName     object
City            object
State           object
Status          object
dtype: object

In [17]:
from sklearn.preprocessing import LabelEncoder

#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['StationId'] = lab.fit_transform(df['StationId'])

In [20]:
df['Date'] = pd.to_numeric(df['Date'], errors='coerce')

In [36]:
df=df.dropna()

In [21]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Date'] = lab.fit_transform(df['Date'])

In [22]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['AQI_Bucket'] = lab.fit_transform(df['AQI_Bucket'])

In [23]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['StationName'] = lab.fit_transform(df['StationName'])

In [24]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['City'] = lab.fit_transform(df['City'])

In [25]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['State'] = lab.fit_transform(df['State'])

In [26]:
#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['Status'] = lab.fit_transform(df['Status'])

In [27]:
from sklearn.model_selection import train_test_split

In [37]:
# Split the dataset into training and testing sets
X = df.drop('AQI', axis=1)
y = df['AQI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from sklearn.tree import DecisionTreeRegressor

In [39]:
rf = RandomForestRegressor()

In [40]:
rf.fit(X_train, y_train)

In [41]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE: ", mse)
print("R-squared: ", r2)

MSE:  83.34398120862389
R-squared:  0.9852801188388209
